## **Problem Scenario: E-commerce Orders Analysis**

### **Background**

You work as a Data Engineer for an e-commerce company. The raw order data contains details about customers, products, payment methods, prices, quantities, and order statuses.

Your team needs to generate daily metrics and product-level insights to help business analysts track performance, identify top products, and analyze customer behavior.

---

## **Business Objectives**

1. **Daily Sales Metrics**

   * Total orders per day
   * Total and average revenue per day
   * Number of unique customers per day
   * Percentage of cancelled orders

2. **Top Products Analysis**

   * Most sold products by quantity
   * Highest revenue products per day
   * Product category sales distribution

3. **Customer Insights**

   * Average order value per customer
   * Total orders per customer
   * Top paying customers

4. **Payment Method Analysis**

   * Count of orders by payment method
   * Revenue split by payment method

---

## **Sample Input Table (Simplified)**

| order\_id | customer\_id | product\_id | category    | payment\_method | order\_time         | price  | quantity | total\_amount | status    |
| --------- | ------------ | ----------- | ----------- | --------------- | ------------------- | ------ | -------- | ------------- | --------- |
| O125678   | C1           | P5          | Electronics | Credit Card     | 2025-08-20 09:15:00 | 250.00 | 2        | 500.00        | Completed |
| O245789   | C2           | P10         | Clothing    | UPI             | 2025-08-20 09:30:00 | 50.00  | 1        | 50.00         | Completed |
| O356890   | C1           | P2          | Electronics | PayPal          | 2025-08-20 10:00:00 | 100.00 | 3        | 300.00        | Cancelled |
| O445678   | C3           | P7          | Books       | Debit Card      | 2025-08-20 11:00:00 | 75.00  | 1        | 75.00         | Completed |

---

## **Expected Output 1 – Daily Metrics Table**

| date       | total\_orders | total\_revenue | avg\_revenue\_per\_order | unique\_customers | cancelled\_pct |
| ---------- | ------------- | -------------- | ------------------------ | ----------------- | -------------- |
| 2025-08-20 | 300           | 45250.00       | 150.83                   | 180               | 8%             |
| 2025-08-21 | 295           | 46300.50       | 156.27                   | 175               | 9%             |

---

## **Expected Output 2 – Top Products Table**

| date       | product\_id | category    | total\_quantity | total\_revenue |
| ---------- | ----------- | ----------- | --------------- | -------------- |
| 2025-08-20 | P5          | Electronics | 35              | 8750.00        |
| 2025-08-20 | P10         | Clothing    | 28              | 1400.00        |
| 2025-08-20 | P2          | Electronics | 20              | 6000.00        |

---

## **Expected Output 3 – Payment Method Analysis**

| date       | payment\_method | order\_count | total\_revenue |
| ---------- | --------------- | ------------ | -------------- |
| 2025-08-20 | Credit Card     | 120          | 25000.00       |
| 2025-08-20 | UPI             | 80           | 10000.00       |
| 2025-08-20 | PayPal          | 50           | 5200.00        |

---

This problem has **real-world complexity** with:

* Missing values in `category` and `payment_method`
* Cancellations impacting revenue
* Daily aggregation needs
* Top product and customer analysis

---

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *

In [9]:
spark = SparkSession.builder.appName("Daily-Day11").getOrCreate()
df = spark.read.option("header", "true").option("inferSchema", "true").csv("ecommerce.csv")
df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- order_time: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- status: string (nullable = true)



In [10]:
df = df.withColumn('date',to_date('order_time','yyyy-MM-dd'))
df.printSchema()
df.show(5, truncate=False)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- order_time: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- status: string (nullable = true)
 |-- date: date (nullable = true)

+--------+-----------+----------+-----------+--------------+-------------------+------+--------+------------+---------+----------+
|order_id|customer_id|product_id|category   |payment_method|order_time         |price |quantity|total_amount|status   |date      |
+--------+-----------+----------+-----------+--------------+-------------------+------+--------+------------+---------+----------+
|O898230 |C187       |P40       |Books      |Debit Card    |2025-08-23 00:06:00|297.45|1       |297.45      |Completed|2025-08-23|
|O483710 |C178      

In [22]:
from pyspark.sql import functions as F

daily_metrics = (
    df.groupBy("date")
      .agg(
          F.countDistinct("order_id").alias("total_orders"),
          F.sum("price").alias("total_revenue"),
          F.avg("price").alias("avg_order_value"),
          F.countDistinct("customer_id").alias("unique_customers"),
          F.countDistinct(F.when(F.col("status") == "Cancelled", F.col("order_id"))).alias("cancelled_orders")
      )
      .withColumn(
          "cancelled_pct",
          (F.col("cancelled_orders") / F.col("total_orders")) * 100
      ).drop("cancelled_orders").orderBy("cancelled_pct", ascending=False)
)
daily_metrics.show(truncate=False)

+----------+------------+-----------------+------------------+----------------+------------------+
|date      |total_orders|total_revenue    |avg_order_value   |unique_customers|cancelled_pct     |
+----------+------------+-----------------+------------------+----------------+------------------+
|2025-08-24|300         |78603.24000000005|262.0108000000002 |154             |12.666666666666668|
|2025-08-20|300         |78358.3          |261.19433333333336|163             |10.333333333333334|
|2025-08-22|300         |77009.09999999999|256.69699999999995|158             |9.666666666666666 |
|2025-08-21|300         |73523.15999999997|245.07719999999992|164             |9.333333333333334 |
|2025-08-23|300         |80900.51999999999|269.66839999999996|167             |7.333333333333333 |
+----------+------------+-----------------+------------------+----------------+------------------+



In [36]:
top_products = df.groupBy("date", "product_id","category") \
    .agg(sum("price").alias("total_revenue"),
        count("product_id").alias("total_quantity")) \
    .withColumn("rank", F.row_number().over(Window.partitionBy("date").orderBy(F.col("total_quantity").desc()))) \
    .filter(F.col("rank") == 1) \
    .orderBy("total_quantity",ascending=False).drop("rank")
top_products.show(truncate=False)

+----------+----------+-----------+------------------+--------------+
|date      |product_id|category   |total_revenue     |total_quantity|
+----------+----------+-----------+------------------+--------------+
|2025-08-24|P23       |Home       |1696.54           |7             |
|2025-08-22|P45       |Electronics|1829.2599999999998|6             |
|2025-08-21|P7        |Clothing   |1114.5            |5             |
|2025-08-23|P35       |Sports     |1377.7200000000003|5             |
|2025-08-20|P26       |Home       |1171.6            |4             |
+----------+----------+-----------+------------------+--------------+



In [40]:
payment_metrics = df.groupBy("date", "payment_method") \
    .agg(
        F.count("order_id").alias("num_transactions"),
        F.sum("price").alias("total_revenue")
    ).orderBy( "total_revenue",ascending=False)
payment_metrics.show(truncate=False)

+----------+--------------+----------------+------------------+
|date      |payment_method|num_transactions|total_revenue     |
+----------+--------------+----------------+------------------+
|2025-08-23|Credit Card   |62              |18107.89          |
|2025-08-20|UPI           |64              |18017.63          |
|2025-08-23|PayPal        |63              |17315.88          |
|2025-08-24|UPI           |59              |17182.589999999997|
|2025-08-20|Credit Card   |68              |17135.48          |
|2025-08-22|Cash          |67              |16972.399999999998|
|2025-08-21|UPI           |66              |16910.809999999994|
|2025-08-24|Cash          |58              |16710.770000000004|
|2025-08-21|Debit Card    |64              |16555.64          |
|2025-08-22|UPI           |63              |16499.84          |
|2025-08-23|Debit Card    |59              |16303.21          |
|2025-08-22|Credit Card   |58              |15984.669999999998|
|2025-08-24|PayPal        |66           

In [43]:
customer_metrics = (
    df.groupBy("customer_id")
      .agg(
          F.countDistinct("order_id").alias("total_orders"),
          F.sum("price").alias("total_spent"),
          F.avg("price").alias("avg_order_value"),
          F.countDistinct("date").alias("active_days")
      )
      .withColumn(
          "avg_orders_per_day",
          F.col("total_orders") / F.col("active_days")
      ).drop("active_days").orderBy("total_spent", ascending=False)
)
customer_metrics.show(5, truncate=False)

+-----------+------------+------------------+------------------+------------------+
|customer_id|total_orders|total_spent       |avg_order_value   |avg_orders_per_day|
+-----------+------------+------------------+------------------+------------------+
|C54        |15          |4721.13           |314.742           |3.0               |
|C94        |17          |4191.120000000001 |246.53647058823535|3.4               |
|C89        |13          |3603.2900000000004|277.17615384615385|2.6               |
|C108       |11          |3456.52           |314.2290909090909 |2.2               |
|C63        |12          |3453.87           |287.8225          |2.4               |
+-----------+------------+------------------+------------------+------------------+
only showing top 5 rows

